In [1]:
import pandas as pd
import re
import openai
import re
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity
import os
from dotenv import load_dotenv
from pathlib import Path  
from azure.core.credentials import AzureKeyCredential  
import inspect
env_path = Path('..') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
openai.api_key =  os.environ.get("AZURE_OPENAI_API_KEY")
openai.api_base =  os.environ.get("AZURE_OPENAI_ENDPOINT")
openai.api_type = "azure"
openai.api_version = "2023-05-15"



### Data generation (Skip this step if data is already generated )

In [2]:
user_message = ""
def generate_airlines():
    user_message =f""" 
    Generate a list of 30 airline company names into a list format with company names separated by commas.
    Just output the data, no need for any comments or explanations as it will break downstream applications.
    The list of the airline company names is:

 
"""

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that knows a lot about the airline industry. You are helping a user find a list of airline company names."},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

airlines = generate_airlines()
airlines

'Airline Company A, Airline Company B, Airline Company C, Airline Company D, Airline Company E, Airline Company F, Airline Company G, Airline Company H, Airline Company I, Airline Company J, Airline Company K, Airline Company L, Airline Company M, Airline Company N, Airline Company O, Airline Company P, Airline Company Q, Airline Company R, Airline Company S, Airline Company T, Airline Company U, Airline Company V, Airline Company W, Airline Company X, Airline Company Y, Airline Company Z, Airline Company AA, Airline Company BB, Airline Company CC.'

In [3]:
airlines = airlines.split(",")
airlines

['Airline Company A',
 ' Airline Company B',
 ' Airline Company C',
 ' Airline Company D',
 ' Airline Company E',
 ' Airline Company F',
 ' Airline Company G',
 ' Airline Company H',
 ' Airline Company I',
 ' Airline Company J',
 ' Airline Company K',
 ' Airline Company L',
 ' Airline Company M',
 ' Airline Company N',
 ' Airline Company O',
 ' Airline Company P',
 ' Airline Company Q',
 ' Airline Company R',
 ' Airline Company S',
 ' Airline Company T',
 ' Airline Company U',
 ' Airline Company V',
 ' Airline Company W',
 ' Airline Company X',
 ' Airline Company Y',
 ' Airline Company Z',
 ' Airline Company AA',
 ' Airline Company BB',
 ' Airline Company CC.']

In [4]:
user_message = ""
def generate_polity_types():
    user_message =f""" 
    Generate a list of policy types for airline industry separated by commas that covers everything from luggage policy to pet policy to food policy to ticket change policy.
    Just output the data, no need for any comments or explanations as it will break downstream applications.
    The list of the policy names is:

 
"""

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that knows a lot about the airline industry. You are helping a user find a list of airline company names."},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

policty_types = generate_polity_types()
policty_types = policty_types.split(",")

policty_types

['Luggage Policy', ' Pet Policy', ' Food Policy', ' Ticket Change Policy']

In [5]:
import time
def generate_policy(airline, policy_type):
    user_message =f""" 
    generate detail content for {policy_type} for {airline} company. 
    Just output the data, no need for any comments or explanations as it will break downstream applications.
    The detail content is:
 
"""
    i=0
    while i<5:
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo", # engine = "deployment_name".
                messages=[
                    {"role": "system", "content": f"You are a helpful AI assistant at {airline} that knows a lot about the airline industry and the company."},
                    {"role": "user", "content":user_message },
                ]
            )
            return response['choices'][0]['message']['content']
        except:
            i+=1
            time.sleep(5)
            


In [6]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector 
import openai
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)  

AZURE_OPENAI_ENDPOINT="https://gpt-4-canadaeast.openai.azure.com/"
AZURE_OPENAI_API_KEY="c1173c2b55054c53b18e9eed1db42bf6"
AZURE_OPENAI_EMB_DEPLOYMENT="text-embedding-ada-002"
AZURE_SEARCH_SERVICE_ENDPOINT="https://kdcogsearch.search.windows.net"
AZURE_SEARCH_ADMIN_KEY="FqALBPgZc3piHQAuoAQgzrwG3t9fhdgUr3FpmuARd8AzSeAFDmPy"

credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
index_name="airline-policy"
# Create a search index
index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="policy_type", type=SearchFieldDataType.String),
    SearchableField(name="policy", type=SearchFieldDataType.String),
    SearchableField(name="airline", type=SearchFieldDataType.String,
                    filterable=True),
    SearchableField(name="active", type=SearchFieldDataType.String,
                    filterable=True),

    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="policy_type"),
        prioritized_keywords_fields=[SemanticField(field_name="policy_type")],
        prioritized_content_fields=[SemanticField(field_name="policy")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')




azcs_search_client = SearchClient(AZURE_SEARCH_SERVICE_ENDPOINT, index_name =index_name , credential=credential)
def generate_embeddings(text):
    openai.api_version = "2023-05-15"
    response = openai.Embedding.create(
        input=text, engine=AZURE_OPENAI_EMB_DEPLOYMENT)
    embeddings = response['data'][0]['embedding']
    return embeddings

def add_or_update(document):
    azcs_search_client.merge_or_upload_documents(documents = document)



 airline-policy created


In [7]:
import uuid
policy_list = []
for airline in airlines[:3]:
    for policty_type in policty_types:
        policy = generate_policy(airline, policty_type)
        policy_type_emb = generate_embeddings(policty_type)
        policy_emb = generate_embeddings(policy)
        policy_document= {"id" : str(uuid.uuid4()), "airline": airline, "policy_type": policty_type, "policy": policy,"titleVector":policy_type_emb, "contentVector":policy_emb, "active": "True"}
        policy_list.append(policy_document)
        

In [8]:
policy_list = policy_list

In [9]:
add_or_update(policy_list)


In [10]:
import os
folder_path ="data"
os.makedirs(folder_path, exist_ok=True)
i=0
articles ={}
file_name="policy_vector.json"
with open(os.path.join(folder_path,file_name), 'w') as file:
        file.write(str(policy_list))




#Test searching

In [11]:
# Semantic Hybrid Search
query = "tell me about checked-in luggage policy"

vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  

results = azcs_search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["policy_type", "policy", "airline"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:

    print(f"Search Reranked Score: {result['@search.reranker_score']}")
    print(f"Search Score: {result['@search.score']}")

    print(f"Title: {result['policy_type']}")
    print(f"Content: {result['policy']}")
    print(f"Category: {result['airline']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Semantic Answer: Checked Baggage Policy: -<em> The number of checked bags and the allowed weight depend on the passenger's class of service and fare type.</em> - For Economy class passengers on domestic flights, the allowance is usually one checked bag, weighing up to 50 pounds (23 kg).
Semantic Answer Score: 0.84814453125

Search Reranked Score: 3.210324764251709
Search Score: 0.032786883413791656
Title: Luggage Policy
Content: Carry-on Baggage Policy:
- Each passenger is allowed to bring one carry-on bag and one personal item (such as a purse, laptop bag, or small backpack) on board.
- The carry-on bag should not exceed the dimensions of 22 x 14 x 9 inches (56 x 36 x 23 cm) and should fit in the overhead bin or under the seat in front of you.
- The personal item should be smaller in size and should fit under the seat in front of you.
- The combined weight of both the carry-on bag and personal item should not exceed 15 pounds (7 kg).

Checked Baggage Policy:
- The number of checked ba

In [12]:
filter_val = 'AA-106'
filter=f"flight_num eq '{filter_val}'"
azcs_search_client2 = SearchClient(AZURE_SEARCH_SERVICE_ENDPOINT, index_name ="flights" , credential=credential)
results = azcs_search_client2.search(  
    search_text=None,
    filter=filter,
    # select=["flight_num"],
    top=3
)

for result in results:

    print(f"Search Reranked Score: {result['@search.reranker_score']}")
    print(f"Search Score: {result['@search.score']}")
    print(result)

ResourceNotFoundError: () The index 'flights' for service 'kdcogsearch' was not found.
Code: 
Message: The index 'flights' for service 'kdcogsearch' was not found.